In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

from glob import glob
from tqdm import tqdm

In [ ]:
village_data = './data/kaggle_public_plant-village/PlantVillage/*'

# custom dataset 정의